In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [19]:
vacation_path = "../output_data/cities.csv"
vacation_df = pd.read_csv(vacation_path)
vacation_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [113]:
gmaps.configure(api_key=g_key)
vac_loc = vacation_df[["Lat", "Lng"]].astype(float)
vac_hum = vacation_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(vac_loc, weights=vac_hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
plt.savefig("../output_data/heatmap1.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [63]:
vacation_perf = vacation_df[vacation_df["Max Temp"] > 68]
vacation_perf = vacation_perf[vacation_perf["Max Temp"] < 72]
vacation_perf = vacation_perf[vacation_perf["Wind Speed"] < 10]
vacation_perf = vacation_perf[vacation_perf["Cloudiness"] < 3]
vacation_perf

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
338,338,port shepstone,2,ZA,1558378854,47,-30.74,30.45,69.54,5.32
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24


In [79]:
hotel_df = vacation_perf
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""
hotelclean_df = hotel_df[["City", "Country", "Lat", "Lng", "Hotel Name", "Hotel Lat", "Hotel Lng"]]
hotelclean_df

,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
259,hami,CN,42.84,93.51,,,
338,port shepstone,ZA,-30.74,30.45,,,
444,birjand,IR,32.86,59.22,,,
533,hakkari,FI,61.34,23.75,,,
538,la seyne-sur-mer,FR,43.10,5.88,,,


In [120]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotelclean_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    try:
        hotelclean_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotelclean_df.loc[index, "Hotel Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotelclean_df.loc[index, "Hotel Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [121]:
hotelclean_df

,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
259,hami,CN,42.84,93.51,Post And Telecommunication Hotel,42.8485,93.505
338,port shepstone,ZA,-30.74,30.45,My Den Beachfront B&B And Self-Catering,-30.7609,30.4457
444,birjand,IR,32.86,59.22,تاکسی سرویس آرامش,32.8625,59.2038
533,hakkari,FI,61.34,23.75,Scandic Eden Nokia,61.468,23.5313
538,la seyne-sur-mer,FR,43.10,5.88,Mama Shelter Marseille,43.2903,5.38903


In [137]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelclean_df.iterrows()]
locations = hotelclean_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Post And Telecommunication Hotel</dd>\n<dt>City</dt><dd>hami</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>My Den Beachfront B&B And Self-Catering</dd>\n<dt>City</dt><dd>port shepstone</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>تاکسی سرویس آرامش</dd>\n<dt>City</dt><dd>birjand</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Scandic Eden Nokia</dd>\n<dt>City</dt><dd>hakkari</dd>\n<dt>Country</dt><dd>FI</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Mama Shelter Marseille</dd>\n<dt>City</dt><dd>la seyne-sur-mer</dd>\n<dt>Country</dt><dd>FR</dd>\n</dl>\n']

In [142]:
# Create a marker_layer using the poverty list to fill the info box

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()                             
fig.add_layer(markers)
fig.add_layer(heat_layer)
plt.savefig("../output_data/heatmap2.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>